In [112]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

In [113]:
uid_table = "segment.landscape_segment"   # 最終的な対応table

%%slack_notify @yusuke_akada


with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    #まず空tableを作る
    query = (
        f"""
        create table {uid_table}
            (
            smn_uid bigint
            ,company_name string
            ,device string
            )
            
            partitioned by 
            (
            segment_id string
            )
        
        stored as parquet
        """
    )
    
    ir.execute_query_no_ret(query) 

今あるセグメントを確認する

In [114]:
query = (
        f"""
        select 
        segment_id
        ,count(distinct smn_uid) as uu
        from segment.landscape_segment
        group by segment_id
        order by segment_id desc
        """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_seg = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [115]:
df_seg.head()

,segment_id,uu
0,ls0104,2546877
1,ls0103,5661235
2,ls0102,1044230
3,ls0101,29144
4,ls0100,50407


LS0105_従業員5人以上

In [116]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

191111


In [117]:
segment_id = 'ls0105'
print(segment_id)

ls0105


In [118]:
start_date = '2019-10-23'
end_date = '2019-10-25'

In [119]:
%%slack_notify @yusuke_akada
# 親テーブルにInsert

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    query = (
        f"""
        insert overwrite table segment.landscape_segment
        (
        smn_uid
        ,company_name
        ,device
        )
        partition
        (
        segment_id = '{segment_id}'
        )
        select
        distinct smn_uid
        ,company_name
        ,case when user_agent_os_family in ('iOS','Android') then 'SP' ELSE 'PC' END as device
        from 
        segment.landscape_ip_company ls
        inner join dsplog.bid_response bid
        on ls.ip_address = bid.remote_address
        where remote_address is not null
        and concat_ws('-',year,month,day) between '{start_date}' and '{end_date}'
        and smn_uid is not null 
        and ip_address is not null
        and isp_flag not in ('1')
        and cast(employee_range as int) between 2 and 12
        """
    )
    ir.execute_query_no_ret(query) 

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [120]:
query = (
        f"""
        select 
        segment_id
        ,count(distinct smn_uid) as uu
        from segment.landscape_segment
        group by segment_id
        order by segment_id desc
        """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_seg = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [42]:
df_seg.head(10)

,segment_id,uu
0,ls0100,50407
1,ls0099,3817940
2,ls0098,2698665
3,ls0097,2807484
4,ls0096,505777
5,ls0095,125907
6,ls0094,237148
7,ls0093,36745
8,ls0092,7189868
9,ls0091,158414


LS0101_教育・学習支援業

In [43]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

191025


In [44]:
segment_id = 'ls0101'
print(segment_id)

ls0101


In [45]:
start_date = '2019-10-22'
end_date = '2019-10-24'

In [46]:
%%slack_notify @yusuke_akada
# 親テーブルにInsert

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    query = (
        f"""
        insert overwrite table segment.landscape_segment
        (
        smn_uid
        ,company_name
        ,device
        )
        partition
        (
        segment_id = '{segment_id}'
        )
        select
        distinct smn_uid
        ,company_name
        ,case when user_agent_os_family in ('iOS','Android') then 'SP' ELSE 'PC' END as device
        from 
        segment.landscape_ip_company ls
        inner join dsplog.bid_response bid
        on ls.ip_address = bid.remote_address
        where remote_address is not null
        and concat_ws('-',year,month,day) between '{start_date}' and '{end_date}'
        and smn_uid is not null 
        and ip_address is not null
        and isp_flag not in ('1')
        and (
        (biz_category_lv1_code1 in ('O') and biz_category_lv3_code1 in ('822'))
        or (biz_category_lv1_code2 in ('O') and biz_category_lv3_code2 in ('823'))
        or (biz_category_lv1_code3 in ('O') and biz_category_lv3_code3 in ('824'))
        or (biz_category_lv1_code3 in ('O') and biz_category_lv3_code3 in ('829'))
        )
        """
    )
    ir.execute_query_no_ret(query) 

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [47]:
query = (
        f"""
        select 
        segment_id
        ,count(distinct smn_uid) as uu
        from segment.landscape_segment
        group by segment_id
        order by segment_id desc
        """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_seg = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [48]:
df_seg.head(10)

,segment_id,uu
0,ls0101,29144
1,ls0100,50407
2,ls0099,3817940
3,ls0098,2698665
4,ls0097,2807484
5,ls0096,505777
6,ls0095,125907
7,ls0094,237148
8,ls0093,36745
9,ls0092,7189868


今あるセグメントを確認する

In [81]:
query = (
        f"""
        select 
        segment_id
        ,count(distinct smn_uid) as uu
        from segment.landscape_segment
        group by segment_id
        order by segment_id desc
        """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_seg = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [82]:
df_seg.head()

,segment_id,uu
0,ls0095,125907
1,ls0094,237148
2,ls0093,36745
3,ls0092,7189868
4,ls0091,158414


LS0096_従業員10名以下

In [3]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

191002


In [4]:
segment_id = 'ls0096'
print(segment_id)

ls0096


In [5]:
start_date = '2019-09-25'
end_date = '2019-09-27'

In [6]:
%%slack_notify @yusuke_akada
# 親テーブルにInsert

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    query = (
        f"""
        insert overwrite table segment.landscape_segment
        (
        smn_uid
        ,company_name
        ,device
        )
        partition
        (
        segment_id = '{segment_id}'
        )
        select
        distinct smn_uid
        ,company_name
        ,case when user_agent_os_family in ('iOS','Android') then 'SP' ELSE 'PC' END as device
        from 
        segment.landscape_ip_company ls
        inner join dsplog.bid_response bid
        on ls.ip_address = bid.remote_address
        where remote_address is not null
        and concat_ws('-',year,month,day) between '{start_date}' and '{end_date}'
        and smn_uid is not null 
        and ip_address is not null
        and isp_flag not in ('1')
        and cast(employee_range as int) between 1 and 2
        """
    )
    ir.execute_query_no_ret(query) 

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [7]:
query = (
        f"""
        select 
        segment_id
        ,count(distinct smn_uid) as uu
        from segment.landscape_segment
        group by segment_id
        order by segment_id desc
        """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_seg = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [8]:
df_seg.head()

,segment_id,uu
0,ls0096,505777
1,ls0095,125907
2,ls0094,237148
3,ls0093,36745
4,ls0092,7189868


LS0097_小売業

In [9]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

191002


In [10]:
segment_id = 'ls0097'
print(segment_id)

ls0097


In [11]:
start_date = '2019-09-25'
end_date = '2019-09-27'

In [12]:
%%slack_notify @yusuke_akada
# 親テーブルにInsert

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    query = (
        f"""
        insert overwrite table segment.landscape_segment
        (
        smn_uid
        ,company_name
        ,device
        )
        partition
        (
        segment_id = '{segment_id}'
        )
        select
        distinct smn_uid
        ,company_name
        ,case when user_agent_os_family in ('iOS','Android') then 'SP' ELSE 'PC' END as device
        from 
        segment.landscape_ip_company ls
        inner join dsplog.bid_response bid
        on ls.ip_address = bid.remote_address
        where remote_address is not null
        and concat_ws('-',year,month,day) between '{start_date}' and '{end_date}'
        and smn_uid is not null 
        and ip_address is not null
        and isp_flag not in ('1')
        and (biz_category_lv1_code1 in ('I') or biz_category_lv1_code2 in ('I') or biz_category_lv1_code3 in ('I'))
        """
    )
    ir.execute_query_no_ret(query) 

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [13]:
query = (
        f"""
        select 
        segment_id
        ,count(distinct smn_uid) as uu
        from segment.landscape_segment
        group by segment_id
        order by segment_id desc
        """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_seg = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [14]:
df_seg.head()

,segment_id,uu
0,ls0097,2807484
1,ls0096,505777
2,ls0095,125907
3,ls0094,237148
4,ls0093,36745


LS0098_上場企業・指定業種

In [15]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

191002


In [16]:
segment_id = 'ls0098'
print(segment_id)

ls0098


In [17]:
start_date = '2019-09-25'
end_date = '2019-09-27'

In [18]:
%%slack_notify @yusuke_akada
# 親テーブルにInsert

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    query = (
        f"""
        insert overwrite table segment.landscape_segment
        (
        smn_uid
        ,company_name
        ,device
        )
        partition
        (
        segment_id = '{segment_id}'
        )
        select
        distinct smn_uid
        ,company_name
        ,case when user_agent_os_family in ('iOS','Android') then 'SP' ELSE 'PC' END as device
        from 
        segment.landscape_ip_company ls
        inner join dsplog.bid_response bid
        on ls.ip_address = bid.remote_address
        where remote_address is not null
        and concat_ws('-',year,month,day) between '{start_date}' and '{end_date}'
        and smn_uid is not null 
        and ip_address is not null
        and isp_flag not in ('1')
        and cast(listed_code as int) between 1 and 6
        and biz_category_lv1_code1 in ('D','E','I','K','M') 
        and OFFICE_ID is not null 
        """
    )
    ir.execute_query_no_ret(query) 

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [19]:
query = (
        f"""
        select 
        segment_id
        ,count(distinct smn_uid) as uu
        from segment.landscape_segment
        group by segment_id
        order by segment_id desc
        """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_seg = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [20]:
df_seg.head()

,segment_id,uu
0,ls0098,2698665
1,ls0097,2807484
2,ls0096,505777
3,ls0095,125907
4,ls0094,237148


LS0099_大企業（東京都内）

In [23]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

191015


今あるセグメントを確認する

In [25]:
query = (
        f"""
        select 
        segment_id
        ,count(distinct smn_uid) as uu
        from segment.landscape_segment
        group by segment_id
        order by segment_id desc
        """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_seg = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [26]:
df_seg.head()

,segment_id,uu
0,ls0098,2698665
1,ls0097,2807484
2,ls0096,505777
3,ls0095,125907
4,ls0094,237148


In [27]:
segment_id = 'ls0099'
print(segment_id)

ls0099


In [28]:
start_date = '2019-09-30'
end_date = '2019-10-02'

In [29]:
%%slack_notify @yusuke_akada
# 親テーブルにInsert

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    query = (
        f"""
        insert overwrite table segment.landscape_segment
        (
        smn_uid
        ,company_name
        ,device
        )
        partition
        (
        segment_id = '{segment_id}'
        )
        select
        distinct smn_uid
        ,company_name
        ,case when user_agent_os_family in ('iOS','Android') then 'SP' ELSE 'PC' END as device
        from 
        segment.landscape_ip_company ls
        inner join dsplog.bid_response bid
        on ls.ip_address = bid.remote_address
        where remote_address is not null
        and concat_ws('-',year,month,day) between '{start_date}' and '{end_date}'
        and smn_uid is not null 
        and ip_address is not null
        and isp_flag not in ('1')
        and cast(capital_range as int) between 5 and 10
        and cast(employee_range as int) between 8 and 12
        and prefecture_name in ("東京都")
        and OFFICE_ID is not null 
        """
    )
    ir.execute_query_no_ret(query) 

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [30]:
query = (
        f"""
        select 
        segment_id
        ,count(distinct smn_uid) as uu
        from segment.landscape_segment
        group by segment_id
        order by segment_id desc
        """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_seg = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [31]:
df_seg.head()

,segment_id,uu
0,ls0099,3817940
1,ls0098,2698665
2,ls0097,2807484
3,ls0096,505777
4,ls0095,125907


LS0102_建設業

In [51]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

191029


今あるセグメントを確認する

In [52]:
query = (
        f"""
        select 
        segment_id
        ,count(distinct smn_uid) as uu
        from segment.landscape_segment
        group by segment_id
        order by segment_id desc
        """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_seg = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [53]:
df_seg.head()

,segment_id,uu
0,ls0101,29144
1,ls0100,50407
2,ls0099,3817940
3,ls0098,2698665
4,ls0097,2807484


In [54]:
segment_id = 'ls0102'
print(segment_id)

ls0102


In [55]:
start_date = '2019-10-23'
end_date = '2019-10-25'

In [56]:
%%slack_notify @yusuke_akada
# 親テーブルにInsert

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    query = (
        f"""
        insert overwrite table segment.landscape_segment
        (
        smn_uid
        ,company_name
        ,device
        )
        partition
        (
        segment_id = '{segment_id}'
        )
        select
        distinct smn_uid
        ,company_name
        ,case when user_agent_os_family in ('iOS','Android') then 'SP' ELSE 'PC' END as device
        from 
        segment.landscape_ip_company ls
        inner join dsplog.bid_response bid
        on ls.ip_address = bid.remote_address
        where remote_address is not null
        and concat_ws('-',year,month,day) between '{start_date}' and '{end_date}'
        and smn_uid is not null 
        and ip_address is not null
        and isp_flag not in ('1')
        and (biz_category_lv1_code1 in ('D') or biz_category_lv1_code2 in ('D') or biz_category_lv1_code3 in ('D'))
        and OFFICE_ID is not null 
        """
    )
    ir.execute_query_no_ret(query) 

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [57]:
query = (
        f"""
        select 
        segment_id
        ,count(distinct smn_uid) as uu
        from segment.landscape_segment
        group by segment_id
        order by segment_id desc
        """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_seg = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [58]:
df_seg.head()

,segment_id,uu
0,ls0102,1044230
1,ls0101,29144
2,ls0100,50407
3,ls0099,3817940
4,ls0098,2698665


LS0103_製造業・卸売業・宿泊飲食・医療福祉

In [61]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

191030


今あるセグメントを確認する

In [62]:
query = (
        f"""
        select 
        segment_id
        ,count(distinct smn_uid) as uu
        from segment.landscape_segment
        group by segment_id
        order by segment_id desc
        """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_seg = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [63]:
df_seg.head()

,segment_id,uu
0,ls0102,1044230
1,ls0101,29144
2,ls0100,50407
3,ls0099,3817940
4,ls0098,2698665


In [64]:
segment_id = 'ls0103'
print(segment_id)

ls0103


In [65]:
start_date = '2019-10-23'
end_date = '2019-10-25'

In [66]:
%%slack_notify @yusuke_akada
# 親テーブルにInsert

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    query = (
        f"""
        insert overwrite table segment.landscape_segment
        (
        smn_uid
        ,company_name
        ,device
        )
        partition
        (
        segment_id = '{segment_id}'
        )
        select
        distinct smn_uid
        ,company_name
        ,case when user_agent_os_family in ('iOS','Android') then 'SP' ELSE 'PC' END as device
        from 
        segment.landscape_ip_company ls
        inner join dsplog.bid_response bid
        on ls.ip_address = bid.remote_address
        where remote_address is not null
        and concat_ws('-',year,month,day) between '{start_date}' and '{end_date}'
        and smn_uid is not null 
        and ip_address is not null
        and isp_flag not in ('1')
        and (biz_category_lv1_code1 in ('E','I','M','P') or biz_category_lv1_code2 in ('E','I','M','P') or biz_category_lv1_code3 in ('E','I','M','P'))
        and OFFICE_ID is not null 
        """
    )
    ir.execute_query_no_ret(query) 

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [67]:
query = (
        f"""
        select 
        segment_id
        ,count(distinct smn_uid) as uu
        from segment.landscape_segment
        group by segment_id
        order by segment_id desc
        """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_seg = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [68]:
df_seg.head()

,segment_id,uu
0,ls0103,5661235
1,ls0102,1044230
2,ls0101,29144
3,ls0100,50407
4,ls0099,3817940


LS0104_上場企業（東京・神奈川・埼玉）

In [69]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

191105


今あるセグメントを確認する

In [70]:
query = (
        f"""
        select 
        segment_id
        ,count(distinct smn_uid) as uu
        from segment.landscape_segment
        group by segment_id
        order by segment_id desc
        """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_seg = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [71]:
df_seg.head()

,segment_id,uu
0,ls0103,5661235
1,ls0102,1044230
2,ls0101,29144
3,ls0100,50407
4,ls0099,3817940


In [72]:
segment_id = 'ls0104'
print(segment_id)

ls0104


In [73]:
start_date = '2019-10-29'
end_date = '2019-10-31'

In [74]:
%%slack_notify @yusuke_akada
# 親テーブルにInsert

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    query = (
        f"""
        insert overwrite table segment.landscape_segment
        (
        smn_uid
        ,company_name
        ,device
        )
        partition
        (
        segment_id = '{segment_id}'
        )
        select
        distinct smn_uid
        ,company_name
        ,case when user_agent_os_family in ('iOS','Android') then 'SP' ELSE 'PC' END as device
        from 
        segment.landscape_ip_company ls
        inner join dsplog.bid_response bid
        on ls.ip_address = bid.remote_address
        where remote_address is not null
        and concat_ws('-',year,month,day) between '{start_date}' and '{end_date}'
        and smn_uid is not null 
        and ip_address is not null
        and isp_flag not in ('1')
        and prefecture_name in ("東京都","神奈川県","埼玉県")
        and cast(listed_code as int) between 1 and 6
        and OFFICE_ID is not null 
        """
    )
    ir.execute_query_no_ret(query) 

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [75]:
query = (
        f"""
        select 
        segment_id
        ,count(distinct smn_uid) as uu
        from segment.landscape_segment
        group by segment_id
        order by segment_id desc
        """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_seg = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [76]:
df_seg.head()

,segment_id,uu
0,ls0104,2546877
1,ls0103,5661235
2,ls0102,1044230
3,ls0101,29144
4,ls0100,50407


笹木さん要望で再開

In [78]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

191111


今あるセグメントを確認する

In [79]:
query = (
        f"""
        select 
        segment_id
        ,count(distinct smn_uid) as uu
        from segment.landscape_segment
        group by segment_id
        order by segment_id desc
        """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_seg = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [80]:
df_seg.head()

,segment_id,uu
0,ls0104,2546877
1,ls0103,5661235
2,ls0102,1044230
3,ls0101,29144
4,ls0100,50407


LS0025_従業員1-1百人未満（関西圏）

In [97]:
segment_id = 'ls0025'
print(segment_id)

ls0025


In [98]:
start_date = '2019-11-06'
end_date = '2019-11-08'

In [99]:
%%slack_notify @yusuke_akada
# 親テーブルにInsert

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    query = (
        f"""
        insert overwrite table segment.landscape_segment
        (
        smn_uid
        ,company_name
        ,device
        )
        partition
        (
        segment_id = '{segment_id}'
        )
        select
        distinct smn_uid
        ,company_name
        ,case when user_agent_os_family in ('iOS','Android') then 'SP' ELSE 'PC' END as device
        from 
        segment.landscape_ip_company ls
        inner join dsplog.bid_response bid
        on ls.ip_address = bid.remote_address
        where remote_address is not null
        and concat_ws('-',year,month,day) between '{start_date}' and '{end_date}'
        and smn_uid is not null 
        and ip_address is not null
        and isp_flag not in ('1')
        and cast(employee_range as int) between 1 and 6
        and prefecture_name in ("大阪府","京都府","兵庫県","滋賀県","奈良県","和歌山県")
        and OFFICE_ID is not null 
        """
    )
    ir.execute_query_no_ret(query) 

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

LS0026_従業員1百-3百人未満（関西圏）

In [100]:
segment_id = 'ls0026'
print(segment_id)

ls0026


In [101]:
start_date = '2019-11-06'
end_date = '2019-11-08'

In [102]:
%%slack_notify @yusuke_akada
# 親テーブルにInsert

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    query = (
        f"""
        insert overwrite table segment.landscape_segment
        (
        smn_uid
        ,company_name
        ,device
        )
        partition
        (
        segment_id = '{segment_id}'
        )
        select
        distinct smn_uid
        ,company_name
        ,case when user_agent_os_family in ('iOS','Android') then 'SP' ELSE 'PC' END as device
        from 
        segment.landscape_ip_company ls
        inner join dsplog.bid_response bid
        on ls.ip_address = bid.remote_address
        where remote_address is not null
        and concat_ws('-',year,month,day) between '{start_date}' and '{end_date}'
        and smn_uid is not null 
        and ip_address is not null
        and isp_flag not in ('1')
        and cast(employee_range as int) = 7
        and prefecture_name in ("大阪府","京都府","兵庫県","滋賀県","奈良県","和歌山県")
        and OFFICE_ID is not null 
        """
    )
    ir.execute_query_no_ret(query) 

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

LS0027_従業員3百-1千人未満（関西圏）

In [103]:
segment_id = 'ls0027'
print(segment_id)

ls0027


In [104]:
start_date = '2019-11-06'
end_date = '2019-11-08'

In [105]:
%%slack_notify @yusuke_akada
# 親テーブルにInsert

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    query = (
        f"""
        insert overwrite table segment.landscape_segment
        (
        smn_uid
        ,company_name
        ,device
        )
        partition
        (
        segment_id = '{segment_id}'
        )
        select
        distinct smn_uid
        ,company_name
        ,case when user_agent_os_family in ('iOS','Android') then 'SP' ELSE 'PC' END as device
        from 
        segment.landscape_ip_company ls
        inner join dsplog.bid_response bid
        on ls.ip_address = bid.remote_address
        where remote_address is not null
        and concat_ws('-',year,month,day) between '{start_date}' and '{end_date}'
        and smn_uid is not null 
        and ip_address is not null
        and isp_flag not in ('1')
        and cast(employee_range as int) = 8
        and prefecture_name in ("大阪府","京都府","兵庫県","滋賀県","奈良県","和歌山県")
        and OFFICE_ID is not null 
        """
    )
    ir.execute_query_no_ret(query) 

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

LS0028_従業員1千-1万人未満（関西圏）

In [106]:
segment_id = 'ls0028'
print(segment_id)

ls0028


In [107]:
start_date = '2019-11-06'
end_date = '2019-11-08'

In [108]:
%%slack_notify @yusuke_akada
# 親テーブルにInsert

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    query = (
        f"""
        insert overwrite table segment.landscape_segment
        (
        smn_uid
        ,company_name
        ,device
        )
        partition
        (
        segment_id = '{segment_id}'
        )
        select
        distinct smn_uid
        ,company_name
        ,case when user_agent_os_family in ('iOS','Android') then 'SP' ELSE 'PC' END as device
        from 
        segment.landscape_ip_company ls
        inner join dsplog.bid_response bid
        on ls.ip_address = bid.remote_address
        where remote_address is not null
        and concat_ws('-',year,month,day) between '{start_date}' and '{end_date}'
        and smn_uid is not null 
        and ip_address is not null
        and isp_flag not in ('1')
        and cast(employee_range as int) between 9 and 11
        and prefecture_name in ("大阪府","京都府","兵庫県","滋賀県","奈良県","和歌山県")
        and OFFICE_ID is not null 
        """
    )
    ir.execute_query_no_ret(query) 

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

LS0029_従業員1万人以上（関西圏）

In [109]:
segment_id = 'ls0029'
print(segment_id)

ls0029


In [110]:
start_date = '2019-11-06'
end_date = '2019-11-08'

In [111]:
%%slack_notify @yusuke_akada
# 親テーブルにInsert

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    query = (
        f"""
        insert overwrite table segment.landscape_segment
        (
        smn_uid
        ,company_name
        ,device
        )
        partition
        (
        segment_id = '{segment_id}'
        )
        select
        distinct smn_uid
        ,company_name
        ,case when user_agent_os_family in ('iOS','Android') then 'SP' ELSE 'PC' END as device
        from 
        segment.landscape_ip_company ls
        inner join dsplog.bid_response bid
        on ls.ip_address = bid.remote_address
        where remote_address is not null
        and concat_ws('-',year,month,day) between '{start_date}' and '{end_date}'
        and smn_uid is not null 
        and ip_address is not null
        and isp_flag not in ('1')
        and cast(employee_range as int) = 12
        and prefecture_name in ("大阪府","京都府","兵庫県","滋賀県","奈良県","和歌山県")
        and OFFICE_ID is not null 
        """
    )
    ir.execute_query_no_ret(query) 

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [ ]:
query = (
        f"""
        select 
        segment_id
        ,count(distinct smn_uid) as uu
        from segment.landscape_segment
        group by segment_id
        order by segment_id desc
        """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_seg = ir.sql_to_pandas(query)

In [ ]:
df_seg.head()

クロスデバイスする場合

In [63]:
xd_table = "segment.drawbridge_device"

In [64]:
segment_id = 'ls0064'
print(segment_id)

ls0064


In [65]:
%%slack_notify @yusuke_akada
# 親テーブルにInsert

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    query = (
        f"""
        insert overwrite table segment.landscape_segment
        (
        smn_uid
        ,company_name
        ,device
        )
        partition
        (
        segment_id = '{segment_id}_xd'
        )
        select 
        distinct xd2.smn_uid
        ,sd.company_name
        ,xd2.device
        from segment.landscape_segment sd
        left join {xd_table} xd
        on sd.smn_uid = xd.smn_uid
        left join {xd_table} xd2
        on xd.drawbridge_consumer_id = xd2.drawbridge_consumer_id
        where xd2.device in (
        'iphone'
        ,'android_phone'
        ,'windows_phone'
        ,'windows_computer'
        ,'linux_computer'
        ,'mac'
        ,'ipad'
        ,'android_tablet'
        ,'windows_tablet') 
        and sd.smn_uid <> xd2.smn_uid
        and segment_id in ('{segment_id}') 
        limit 5000000
        """
    )
    ir.execute_query_no_ret(query) 

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

In [ ]:
query = (
        f"""
        select 
        segment_id
        ,count(distinct smn_uid) as uu
        from segment.landscape_segment
        group by segment_id
        order by segment_id desc
        """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_seg = ir.sql_to_pandas(query)

In [35]:
df_seg.head(10)

,segment_id,uu
0,ls0085_xd,1145382
1,ls0085,987078
2,ls0084,6997972
3,ls0083,2838589
4,ls0082,3153840
5,ls0081,5819789
6,ls0080,2556255
7,ls0079_xd,1119253
8,ls0079,570221
9,ls0078_xd,1004602
